## Run inference on the Llama 2 endpoint you have created.

In [1]:
import json
import boto3

### Supported Parameters

***
This model supports many parameters while performing inference. They include:

* **max_length:** Model generates text until the output length (which includes the input context length) reaches `max_length`. If specified, it must be a positive integer.
* **max_new_tokens:** Model generates text until the output length (excluding the input context length) reaches `max_new_tokens`. If specified, it must be a positive integer.
* **num_beams:** Number of beams used in the greedy search. If specified, it must be integer greater than or equal to `num_return_sequences`.
* **no_repeat_ngram_size:** Model ensures that a sequence of words of `no_repeat_ngram_size` is not repeated in the output sequence. If specified, it must be a positive integer greater than 1.
* **temperature:** Controls the randomness in the output. Higher temperature results in output sequence with low-probability words and lower temperature results in output sequence with high-probability words. If `temperature` -> 0, it results in greedy decoding. If specified, it must be a positive float.
* **early_stopping:** If True, text generation is finished when all beam hypotheses reach the end of sentence token. If specified, it must be boolean.
* **do_sample:** If True, sample the next word as per the likelihood. If specified, it must be boolean.
* **top_k:** In each step of text generation, sample from only the `top_k` most likely words. If specified, it must be a positive integer.
* **top_p:** In each step of text generation, sample from the smallest possible set of words with cumulative probability `top_p`. If specified, it must be a float between 0 and 1.
* **return_full_text:** If True, input text will be part of the output generated text. If specified, it must be boolean. The default value for it is False.
* **stop**: If specified, it must be a list of strings. Text generation stops if any one of the specified strings is generated.

We may specify any subset of the parameters mentioned above while invoking an endpoint. Next, we show an example of how to invoke endpoint with these arguments.

**NOTE**: If `max_new_tokens` is not defined, the model may generate up to the maximum total tokens allowed, which is 4K for these models. This may result in endpoint query timeout errors, so it is recommended to set `max_new_tokens` when possible. For 7B, 13B, and 70B models, we recommend to set `max_new_tokens` no greater than 1500, 1000, and 500 respectively, while keeping the total number of tokens less than 4K.

***

In [128]:
default_system_prompt = """
Craft a vibrant and engaging product description item whose information is given in the prompt. Transform these features into an alluring narrative that emphasizes the product's unique qualities. Highlight the practicality, quality, and any unique selling points, making it irresistible to potential buyers.
"""

In [129]:
prompts={
        "prompt": answer,
        "system_prompt": default_system_prompt
        }

In [130]:
payloads = []
payloads.append(
    {
        "inputs": prompts, 
        "parameters": {"max_new_tokens": 64, "top_p": 0.9, "temperature": 0, "return_full_text": True},
    }
)

### Query endpoint that you have created

---

In [131]:
endpoint_name = 'jumpstart-dft-meta-textgeneration-llama-2-13b-1'


def query_endpoint(payload):
    client = boto3.client("sagemaker-runtime")
    response = client.invoke_endpoint(
        EndpointName=endpoint_name,
        ContentType="application/json",
        Body=json.dumps(payload),
        CustomAttributes="accept_eula=true",
    )
    response = response["Body"].read().decode("utf8")
    response = json.loads(response)
    return response

In [132]:
for payload in payloads:
    query_response = query_endpoint(payload)
    #print(payload["inputs"])
    print(f"> {query_response[0]['generated_text']}")
    print("\n======\n")

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received client error (422) from primary with message "Failed to deserialize the JSON body into the target type: inputs: invalid type: map, expected a string at line 1 column 11". See https://us-east-1.console.aws.amazon.com/cloudwatch/home?region=us-east-1#logEventViewer:group=/aws/sagemaker/Endpoints/jumpstart-dft-meta-textgeneration-llama-2-13b-1 in account 275461957965 for more information.

In [3]:
import os
import json
import gzip
import pandas as pd
from urllib.request import urlopen

In [4]:
### load the meta data

data = []
with gzip.open('meta_AMAZON_FASHION.json.gz') as f:
    for l in f:
        data.append(json.loads(l.strip()))
    
# total length of list, this number equals total number of products
print(len(data))

# first row of the list
print(data[0])

186637
{'title': 'Slime Time Fall Fest [With CDROM and Collector Cards and Neutron Balls, Incredi-Ball and Glow Stick Necklace, Paper Fram', 'brand': 'Group Publishing (CO)', 'feature': ['Product Dimensions:\n                    \n8.7 x 3.6 x 11.4 inches', 'Shipping Weight:\n                    \n2.4 pounds'], 'rank': '13,052,976inClothing,Shoesamp;Jewelry(', 'date': '8.70 inches', 'asin': '0764443682', 'imageURL': ['https://images-na.ssl-images-amazon.com/images/I/51bSrINiWpL._US40_.jpg'], 'imageURLHighRes': ['https://images-na.ssl-images-amazon.com/images/I/51bSrINiWpL.jpg']}


In [5]:
# convert list into pandas dataframe

df = pd.DataFrame.from_dict(data)

print(len(df))

186637


In [6]:
# Filter rows where column 'A' is not null
filtered_df = df[df['description'].notnull()]

In [7]:
# List all columns
column_names = df.columns

print(column_names)

Index(['title', 'brand', 'feature', 'rank', 'date', 'asin', 'imageURL',
       'imageURLHighRes', 'description', 'price', 'also_view', 'also_buy',
       'fit', 'details', 'similar_item', 'tech1'],
      dtype='object')


In [8]:
filtered_df = filtered_df[['title', 'brand', 'feature', 'description', 'price']]

In [9]:
filtered_df = filtered_df.dropna(how='any')

In [10]:
filtered_df

,title,brand,feature,description,price
17,"X. L. Carbon Fiber Money Clip, made in the USA",Roar Carbon,"[Real Carbon Fiber, Made in USA, 5 year warran...",[When you pull out your extra large carbon fib...,$14.99
18,Shimmer Anne Shine Clip On Costume/Halloween C...,Shimmer Anne Shine,[Shimmer Anne Shine Clip On Costume/Halloween ...,"[A fun addition to any costume party, play, or...",$6.99
69,Buxton Heiress Pik-Me-Up Framed Case,Buxton,"[Leather, Imported, synthetic lining, Flap clo...",[Authentic crunch leather with rich floral emb...,$16.95
331,Art Nouveau Sterling Silver Ornate Repousse He...,Silver Insanity,"[2&5/8"" High and 3/4"" Wide, Weight is Approx. ...","[It measures 2&5/8"" tall x 3/4"" wide, weighs a...",$44.66
410,Dream PJ's Blue - Large - Part #: 25BLG,Ethical/Spot,[Product Dimensions:\n \n8....,"[SOFT AND CUDDLY, SWEET DREAM PAJAMAS IN SOFT ...",$15.99
...,...,...,...,...,...
186502,Women's Roman Empress Costume - Greek Goddess ...,Karnival Costumes,"[100% polyester, CAN BE USED YEAR ROUND: These...",[Look great in a durable and high quality cost...,$24.99
186506,"Women's Sexy Gladiator Costume, for Halloween ...",Karnival Costumes,"[100% polyester, CAN BE USED YEAR ROUND: These...",[Look great in a durable and high quality cost...,$20.99
186538,Georgia Boot AMP Insole,Georgia Boot,"[NA, Imported, Memory foam adjusts for customi...",[Ready to finally be comfortable in your boots...,$21.27
186552,Single Flare Steel Plugs with Mint Green Rose ...,Pierced Owl,[Pair of single flare steel plugs with mint gr...,[Single Flare Steel Plugs with Mint Green Rose...,$18.99


In [120]:
def create_description_prompts(df):
    prompts = []
    
    for index, row in df.iterrows():
        feature_list = '\n  - '.join(row['feature'])
        prompt = (
            f"Title: {row['title']}\n"
            f"Brand: {row['brand']}\n"
            f"Price: ${row['price']}\n"
            f"Key Features: \n  - {feature_list}\n\n"
        )
        prompts.append(prompt)

    return prompts

In [121]:
head = filtered_df.head(1)

In [122]:
# Generate the prompts for each product
prompts = create_description_prompts(head)
answer = ''

# Print each prompt
for i, prompt in enumerate(prompts, 1):
    answer = f"{prompt}\n"

In [123]:
print(answer)

Title: X. L. Carbon Fiber Money Clip, made in the USA
Brand: Roar Carbon
Price: $$14.99
Key Features: 
  - Real Carbon Fiber
  - Made in USA
  - 5 year warranty against delamination
  - Holds Cash and Cards
  - You can return this item for any reason and get a full refund: no shipping charges. The item must be returned in new and unused condition.
  - Read the full returns policy
  - Go to Your Orders to start the return
  - Print the return shipping label
  - Ship it!
  - Package Dimensions:
                    
2.5 x 2.2 x 0.6 inches
  - Shipping Weight:
                    
0.32 ounces (View shipping rates and policies)



